# Make tensors (time, location, age; count)

In [14]:
import datetime
import os
import shutil
import warnings
import pandas as pd
import numpy as np
import tqdm

In [15]:
ROOTPATH = "../dat/raw/"
OUTPATH = "../dat/"
os.makedirs(OUTPATH, exist_ok=True)
warnings.filterwarnings("ignore")

In [16]:
def load_data(date_index):
    return pd.read_csv(ROOTPATH + date_index + "-citibike-tripdata.csv")

In [26]:
def generate_station_data(start="2017-01", end="2020-12", outpath=None):
    """ Extract the consisten station information
        between 'start' and 'end'.
    """

    df_station_list = []

    for date_index in tqdm.tqdm(pd.date_range(start="2016-01", end="2020-12", freq='m')):

        # original trip data
        df = load_data(date_index.strftime("%Y%m"))

        # ensure the consistency of column names
        df.columns = df.columns.str.replace(" ", "")
        df.columns = df.columns.str.lower()

        # start station data
        df_stt = df[[
            "startstationid",
            "startstationname",
            "startstationlatitude",
            "startstationlongitude"
        ]]

        # end station data
        df_end = df[[
            "endstationid",
            "endstationname",
            "endstationlatitude",
            "endstationlongitude"
        ]]

        # rename columns
        df_stt.columns = df_stt.columns.str.replace("start", "")
        df_end.columns = df_end.columns.str.replace("end", "")

        # get unique station data
        df = pd.concat([df_stt, df_end])
        df = df.drop_duplicates(subset=["stationid"])

        df_station_list.append(df.copy())

    # extract full station data
    station_data = pd.concat(df_station_list)
    station_data = station_data.drop_duplicates(subset=["stationid"])
    station_data = station_data.dropna(how="any")
    station_data = station_data.sort_values("stationname")
    station_data = station_data.reset_index(drop=True)  # remove unused index
    station_data = station_data.reset_index(drop=True)  # remove unused index
    station_data = station_data.reset_index()  # save unique id from zero
    station_data = station_data.astype({"stationid": int})

    if outpath is not None:
        station_data.to_csv(
            os.path.join(outpath, "station_data.csv"),
            index=False)

    return station_data

In [27]:
# station_data = generate_station_data(start="2017-01", end="2020-12", outpath=OUTPATH)

In [28]:
# station_data.reset_index(drop=True).reset_index()
# station_data.stationid.sort_values().reset_index(drop=True).plot()

In [29]:
def process(df):
    """ Load $ preprocess the original trip data """
    # Cleaning
    df = df.dropna(how="any")
    df.columns = df.columns.str.replace(" ", "")
    df.columns = df.columns.str.lower()
    df = df.query("usertype=='Subscriber'")
    df["userage"] = 2013 - df["birthyear"]
    df = df[["starttime", "startstationid", "endstationid", "userage"]]
    df = df.reset_index(drop=True)

    # Cast
    df = df.astype({
        "startstationid": int,
        "endstationid": int,
        "userage": int
    })
    
    return df

In [30]:
def save_dataset(start="2017-01", end="2020-12"):

    dataset_path = OUTPATH + "citibike_{}_{}/".format(
        start.replace("-", ""), end.replace("-", ""))
    if os.path.exists(dataset_path):
        shutil.rmtree(dataset_path)
    os.makedirs(dataset_path)

    # Get Station Information
    station_data = generate_station_data(start, end, dataset_path)

    # Make Monthly Trip History
    for date_index in pd.date_range(start=start, end=end, freq='m'):
        # print(date_index)

        # data processing
        df = load_data(date_index.strftime("%Y%m"))
        df = process(df)

        # encode start station indices
        df = pd.merge(df, station_data[["index", "stationid"]],
                      how="left",
                      left_on="startstationid",
                      right_on="stationid")

        df = df.rename({"index": "start_station_dim"}, axis=1)
        del df["stationid"]

        # encode end station indices
        df = pd.merge(df, station_data[["index", "stationid"]],
                      how="left",
                      left_on="endstationid",
                      right_on="stationid")

        df = df.rename({"index": "end_station_dim"}, axis=1)
        del df["stationid"]

        # cleaning
        df = df.dropna(how="any")

        # save
        # print(df.head())
        filename = "tripdata_{}.csv".format(date_index.strftime("%Y%m"))
        print("[SAVED]", filename)
        df.to_csv(dataset_path + filename, index=False)

In [31]:
save_dataset(start="2019-03", end="2021-04")  # 2021-04 will be excluded
# save_dataset(start="2017-01", end="2021-01")

100%|██████████| 59/59 [02:39<00:00,  2.71s/it]
[SAVED] tripdata_201903.csv
[SAVED] tripdata_201904.csv
[SAVED] tripdata_201905.csv
[SAVED] tripdata_201906.csv
[SAVED] tripdata_201907.csv
[SAVED] tripdata_201908.csv
[SAVED] tripdata_201909.csv
[SAVED] tripdata_201910.csv
[SAVED] tripdata_201911.csv
[SAVED] tripdata_201912.csv
[SAVED] tripdata_202001.csv
[SAVED] tripdata_202002.csv
[SAVED] tripdata_202003.csv
[SAVED] tripdata_202004.csv
[SAVED] tripdata_202005.csv
[SAVED] tripdata_202006.csv
[SAVED] tripdata_202007.csv
[SAVED] tripdata_202008.csv
[SAVED] tripdata_202009.csv
[SAVED] tripdata_202010.csv
[SAVED] tripdata_202011.csv
[SAVED] tripdata_202012.csv
[SAVED] tripdata_202101.csv
[SAVED] tripdata_202102.csv
[SAVED] tripdata_202103.csv
